In [14]:
from huggingface_hub import InferenceClient


In [48]:
data = [
# 1. Сильный рост по всем фронтам (День)
{
  "period": "2023-11-15",
  "comparison_period": "2023-11-14",
  "metrics": {
    "GMV": {"value": 1625000, "change": 0.25}, # +25%
    "Заказы": {"value": 2400, "change": 0.20},  # +20%
    "Средний чек (AOV)": {"value": 677.08, "change": 0.042}, # +4.2%
    "Сессии": {"value": 60000, "change": 0.18},  # +18%
    "Конверсия (CR)": {"value": 0.04, "change": 0.017} # +1.7%
  }
},
# 2. Сильное падение по всем фронтам (День)
{
  "period": "2023-11-16",
  "comparison_period": "2023-11-15",
  "metrics": {
    "GMV": {"value": 980000, "change": -0.22},  # -22%
    "Заказы": {"value": 1800, "change": -0.18},  # -18%
    "Средний чек (AOV)": {"value": 544.44, "change": -0.049},# -4.9%
    "Сессии": {"value": 48000, "change": -0.15},  # -15%
    "Конверсия (CR)": {"value": 0.0375, "change": -0.035}# -3.5%
  }
},
# 3. Смешанная динамика: GMV растет, Заказы падают (День) -> Рост за счет AOV
{
  "period": "2023-11-17",
  "comparison_period": "2023-11-16",
  "metrics": {
    "GMV": {"value": 1060000, "change": 0.082}, # +8.2%
    "Заказы": {"value": 1710, "change": -0.05},  # -5%
    "Средний чек (AOV)": {"value": 619.88, "change": 0.139}, # +13.9%
    "Сессии": {"value": 49000, "change": 0.021},  # +2.1%
    "Конверсия (CR)": {"value": 0.0349, "change": -0.069} # -6.9%
  }
},

# 4. Смешанная динамика: GMV падает, Заказы растут (День) -> Падение за счет AOV
{
  "period": "2023-11-18",
  "comparison_period": "2023-11-17",
  "metrics": {
    "GMV": {"value": 985800, "change": -0.07},  # -7%
    "Заказы": {"value": 1778, "change": 0.04},  # +4%
    "Средний чек (AOV)": {"value": 554.44, "change": -0.106},# -10.6%
    "Сессии": {"value": 51000, "change": 0.041},  # +4.1%
    "Конверсия (CR)": {"value": 0.03486, "change": -0.001}# -0.1%
  }
},

# 5. Почти без изменений / Стабильность (День)
{
  "period": "2023-11-19",
  "comparison_period": "2023-11-18",
  "metrics": {
    "GMV": {"value": 990000, "change": 0.004}, # +0.4%
    "Заказы": {"value": 1785, "change": 0.004}, # +0.4%
    "Средний чек (AOV)": {"value": 554.62, "change": 0.0},  # 0%
    "Сессии": {"value": 51100, "change": 0.002}, # +0.2%
    "Конверсия (CR)": {"value": 0.03493, "change": 0.002} # +0.2%
  }
},

# 6. Умеренный недельный рост
{
  "period": "Неделя 46, 2023",
  "comparison_period": "Неделя 45, 2023",
  "metrics": {
    "GMV": {"value": 7800000, "change": 0.09},  # +9%
    "Заказы": {"value": 13500, "change": 0.07},  # +7%
    "Средний чек (AOV)": {"value": 577.78, "change": 0.019}, # +1.9%
    "Сессии": {"value": 450000, "change": 0.05},  # +5%
    "Конверсия (CR)": {"value": 0.03, "change": 0.019} # +1.9%
  }
},

# 7. Недельное падение с аномалией в трафике (GMV/Заказы вниз, Сессии вверх)
{
  "period": "Неделя 47, 2023",
  "comparison_period": "Неделя 46, 2023",
  "metrics": {
    "GMV": {"value": 7332000, "change": -0.06}, # -6%
    "Заказы": {"value": 12420, "change": -0.08}, # -8%
    "Средний чек (AOV)": {"value": 590.34, "change": 0.022}, # +2.2%
    "Сессии": {"value": 472500, "change": 0.05},  # +5% (аномалия!)
    "Конверсия (CR)": {"value": 0.0263, "change": -0.124} # -12.4%
  }
},

# 8. Фокус на конверсии (День) (GMV/Заказы стабильны, Сессии упали -> CR выросла)
{
  "period": "2023-11-20",
  "comparison_period": "2023-11-19",
  "metrics": {
    "GMV": {"value": 995000, "change": 0.005}, # +0.5%
    "Заказы": {"value": 1790, "change": 0.003}, # +0.3%
    "Средний чек (AOV)": {"value": 555.87, "change": 0.002}, # +0.2%
    "Сессии": {"value": 48545, "change": -0.05}, # -5%
    "Конверсия (CR)": {"value": 0.03687, "change": 0.056} # +5.6%
  }
},

# 9. Рост за счет AOV (Неделя) (GMV умеренно вырос, Заказы слегка упали)
{
  "period": "Неделя 48, 2023",
  "comparison_period": "Неделя 47, 2023",
  "metrics": {
    "GMV": {"value": 7845240, "change": 0.07},  # +7%
    "Заказы": {"value": 12234, "change": -0.015}, # -1.5%
    "Средний чек (AOV)": {"value": 641.25, "change": 0.086}, # +8.6%
    "Сессии": {"value": 470000, "change": -0.005}, # -0.5%
    "Конверсия (CR)": {"value": 0.0260, "change": -0.01} # -1%
  }
},

# 10. Резкий всплеск (например, старт большой распродажи) (День)
{
  "period": "2023-11-24", # Черная пятница :)
  "comparison_period": "2023-11-23",
  "metrics": {
    "GMV": {"value": 2400000, "change": 0.6},  # +60%
    "Заказы": {"value": 3100, "change": 0.55},  # +55%
    "Средний чек (AOV)": {"value": 774.19, "change": 0.032}, # +3.2%
    "Сессии": {"value": 80000, "change": 0.4},  # +40%
    "Конверсия (CR)": {"value": 0.03875, "change": 0.107} # +10.7%
  }
}
]

In [62]:
API_TOKEN = 'some_token'

In [63]:
def qa_kpi_llama32(data):
    client = InferenceClient("meta-llama/Llama-3.1-405B-Instruct", token=API_TOKEN)
    for kpi in data:
        messages = [
            {"role": "system", "content": "Ты бизнес-аналитик, делаешь краткие сводки по метрикам. На основе данных ниже, составь краткую связную сводку (2-4 предложения) на русском языке для менеджера. Опиши динамику ключевой метрики GMV и других важных изменений. Используй только предоставленные данные, не добавляй ничего лишнего."},
            {"role": "user", "content": f"Составь краткую сводку по метрикам на основе данных. Изменения (changes) указаны не в процентах, поэтому домножь их на 100 и поставь '%'. Вот сами данные: {kpi}."}
        ]
        choices = client.chat_completion(messages, max_tokens=500)
        print(kpi['period'])
        print(choices.choices[0].message.content)
        print('\n')

In [64]:
qa_kpi_llama32(data)

2023-11-15
Сводка по ключевым метрикам за 15 ноября 2023 года по сравнению с 14 ноября 2023 года:

GMV увеличился на 25% и составил 1 625 000. Количество заказов выросло на 20% и достигло 2400. Средний чек (AOV) незначительно увеличился на 4,2% и составил 677,08. Общее количество сессий увеличилось на 18% и достигло 60 000. Конверсия (CR) улучшилась на 1,7% и составила 0,04.


2023-11-16
За период с 15 по 16 ноября 2023 года GMV снизился на 22% до 980 000. Заказы уменьшились на 18% и составили 1800. Средний чек сократился на 4,9% и составил 544,44, а конверсия снизилась на 3,5% до 3,75%.


2023-11-17
В период с 16 по 17 ноября 2023 года GMV вырос на 8,2% и составил 1 060 000. При этом наблюдалось снижение количества заказов на 5%. Средний чек увеличился на 13,9% и составил 619,88, а количество сессий возросло на 2,1%. Однако конверсия снизилась на 6,9% до 3,49%.


2023-11-18
За период с 17 по 18 ноября 2023 года GMV снизился на 7%. При этом наблюдается увеличение количества заказов на 